# Classification of images

Given are images that represent the hand gestures of the game "paper, rock, scissors" and that are stored in the respective
folders. In an additionally folder random images (that do not correspond to one of these gestures) are stored.
A classification model is to be build that classifies an image to one of the four labels "paper", "rock", "scissors" or "rest".

This project was created by Julian Kartte. For a more detailed description check out my [github](https://github.com/juliankartte/showroom). In case you are interested in getting the date used in this project feel free to contact me on [linkedin](https://www.linkedin.com/in/julian-kartte-aa64a9237/).

In [ ]:
import sys
import tensorflow as tf
import numpy as np
import shutil

from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ReduceLROnPlateau

from tensorflow.python.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import os
import csv
from datetime import date
import pandas as pd
import random
import itertools
import json
from tensorflow.keras.constraints import max_norm

# Configuration

**List of included parameters**:
- **x_scale**: Number of pixels on the x-axis
- **y_scale**: Number of pixels on the y-axis
- **validation_split**: Ration of data in validation set


- **use_data_preprocessing**: Doubles the amount of input-data bei rotating them by 180 degrees
- **use_data_augmentation**: Use data augmentation in keras ImageGenerators


- **modelname**: select one of the following models
 - FarahAmalia (acc=96): https://medium.com/geekculture/rock-paper-scissors-image-classification-using-cnn-eefe4569b415
 - Devakumar (acc=90): https://www.kaggle.com/code/imdevskp/rock-paper-scissors-image-classification-using-cnn/notebook
 - Aditya (acc=100): https://www.kaggle.com/code/recursion17/rockpaperscissors-100-accuracy
 - Custom: custom model
 - Custom_constraints: custom model with constraints
 
 
- **optimizer**: select one of the following optimizers
 - Adam
 - Adadelta
 - Adagrad
 - RMSprop
 - SGD


- **loss_function**: sets the loss_function of the model
- **epochs**: number of epochs
- **batch_size**: sets the batch size
- **dropout_ration**: sets the dropout_ratio of dropout_layers


- **learning_rate_reduction**: Determines whether to use learning rate reduction on plateau or not.
- **learning_rate_monitor**: monitored quantity for learning_rate_reduction
- **learning_rate_patience**: number of epochs to wait
- **learning_rate_verbose**: update messages
- **learning_rate_factor**: ration of reduction
- **learnint_rate_start_lr**: sets starting learning rate
- **learning_rate_min_lr**: minimal learning rate


- **early_stopping**: Use early stopping or not
- **early_stopping_monitor**: monitored quantity
- **early_stopping_min_delta**: minimal difference between quantity needed to be seen as a decrease
- **early_stopping_patience**: number of rounds waiting since last decrease before stopping
- **early_stopping_verbose**: output messages
- **early_stopping_restore_best_weights**: restore best weights after early stopping


- **use_kaggle_data**: uses provided data
- **use_grayscaled_data**: uses provided data turned into grayscale
- **use_grayscaled_augmented_data**: uses augmented data turned into grayscale
- **use_rest_class**: uses rest class or not

In [ ]:
gs_config = {
    'x_scale': [30], # 60, 120
    'y_scale': [20], # 40, 80
    'validation_split': [0.2],
    
    'use_data_augmentation': [True],
    
    'modelname': ['Custom_constraints'], #'Custom', 'Aditya', 'FarahAmalia', 'Devakumar'],
    'optimizer': [
        tf.keras.optimizers.Adam(),
        #tf.keras.optimizers.Adadelta(),
        #tf.keras.optimizers.Adagrad(),
        #tf.keras.optimizers.Adamax(),
        #tf.keras.optimizers.RMSprop(),
        #tf.keras.optimizers.SGD()
        ],
    
    'loss_function': ['categorical_crossentropy'],
    'epochs': [50],
    'batch_size': [16],
    'dropout_ratio': [0.3],

    'learning_rate_reduction': [True],
    'learning_rate_monitor': ['val_categorical_accuracy'],
    'learning_rate_patience': [2],
    'learning_rate_verbose': [1],
    'learning_rate_factor': [0.25],
    'learning_rate_start_lr': [0.001],
    'learning_rate_min_lr': [0.000003],

    'early_stopping': [True],
    'early_stopping_monitor': ['val_categorical_accuracy'], #'val_loss'],
    'early_stopping_min_delta': [0],
    'early_stopping_patience': [10],
    'early_stopping_verbose': [1],
    'early_stopping_restore_best_weights': [True],

    'use_grayscaled_data': [False],
}

# Data augmentation and preprocessing

In [ ]:
def get_data(input_dict):
    """
    Loads the images from the path './Traindata' according to the configuration in gs_config.
    Returns train_generator, validation_generator.
    """
    
    x_scale: int = input_dict['x_scale']
    y_scale: int = input_dict['y_scale']
    batch_size: int = input_dict['batch_size']
    data_augmentation: bool = input_dict['use_data_augmentation']
    validation_split: float = input_dict['validation_split']
    use_grayscaled_data : bool = input_dict['use_grayscaled_data']
        
    if use_grayscaled_data:
        color_mode = 'grayscale'
    else:
        color_mode = 'rgb'

    if data_augmentation :
        datagen = ImageDataGenerator(rescale = 1.0/255,
                                     rotation_range = 20,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     fill_mode = 'nearest',
                                     validation_split = validation_split
                              )
        valgen = ImageDataGenerator(rescale = 1.0/255,
                                    validation_split = validation_split)
    else:
        datagen = ImageDataGenerator(rescale = 1.0/255,
                                     validation_split = validation_split
                                    )
        valgen = ImageDataGenerator(rescale = 1.0/255,
                                     validation_split = validation_split
                                   )

    train_generator = datagen.flow_from_directory(
            './Traindata',
            target_size=(x_scale, y_scale),
            color_mode = color_mode,
            batch_size=batch_size,
            shuffle=True,
            class_mode='categorical',
            subset='training',
    )
    
    validation_generator = valgen.flow_from_directory(
            './Traindata', 
            target_size=(x_scale, y_scale),
            color_mode = color_mode,
            batch_size=batch_size,
            shuffle=True,
            class_mode='categorical',
            subset='validation')

    return train_generator, validation_generator

# Creating and fitting the models

In [ ]:
def get_model(optimizer, labels_count, input_dict):    
    """
    Builds a model according to the specification in the current iteration input_dict of gs_config.
    Returns the built model.
    """
    
    modelname = input_dict['modelname']
    loss_function = input_dict['loss_function']
    x_scale = input_dict['x_scale']
    y_scale = input_dict['y_scale']
    lr_start = input_dict['learning_rate_start_lr']
    dropout_ratio = input_dict['dropout_ratio']
    
    if input_dict['use_grayscaled_data']:
        color_layer = 1
    else:
        color_layer = 3
    
    if modelname == 'FarahAmalia':
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(x_scale, y_scale, color_layer)),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, padding = 'Same'),
            tf.keras.layers.MaxPooling2D(2,2),

            tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu, padding = 'Same'),
            tf.keras.layers.MaxPooling2D(2,2),

            tf.keras.layers.Flatten(),

            tf.keras.layers.Dense(256, activation=tf.nn.relu),
            tf.keras.layers.Dense(labels_count, activation = tf.nn.softmax),
        ])
        
    elif modelname == 'Devakumar':
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(x_scale, y_scale, color_layer)),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(labels_count, activation='sigmoid'),
        ])
        
    elif modelname == 'Aditya':
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (5,5), activation=tf.nn.relu, input_shape=(x_scale, y_scale, color_layer)),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, padding = 'Same'),
            tf.keras.layers.MaxPooling2D(2,2),

            tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu, padding = 'Same'),
            tf.keras.layers.MaxPooling2D(2,2),

            tf.keras.layers.Flatten(),

            tf.keras.layers.Dense(256, activation=tf.nn.relu),
            tf.keras.layers.Dense(labels_count, activation = tf.nn.softmax)
        ])

    elif modelname == 'Custom':
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (5,5), activation=tf.nn.relu, input_shape=(x_scale, y_scale, color_layer)),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, padding = 'Same'),
            tf.keras.layers.MaxPooling2D(2,2),

            tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu, padding = 'Same'),
            tf.keras.layers.MaxPooling2D(2,2),

            tf.keras.layers.Flatten(),

            tf.keras.layers.Dense(256, activation=tf.nn.relu),
            tf.keras.layers.Dense(labels_count, activation = tf.nn.softmax)
        ])
        
    elif modelname == 'Custom_constraints':
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, kernel_constraint=max_norm(3), 
                                   input_shape=(x_scale, y_scale, color_layer)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(dropout_ratio),

            tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, padding = 'Same', kernel_constraint=max_norm(3)),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Dropout(dropout_ratio),

            tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu, padding = 'Same', kernel_constraint=max_norm(3)),
            tf.keras.layers.MaxPooling2D(2,2),
            tf.keras.layers.Dropout(dropout_ratio),

            tf.keras.layers.Flatten(),

            tf.keras.layers.Dense(256, activation=tf.nn.relu),
            tf.keras.layers.Dense(labels_count, activation = tf.nn.softmax),
        ])
    
    temp_metrics = [
        tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
        tf.keras.metrics.BinaryAccuracy(name='binary_accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

    if type(optimizer) == type(tf.keras.optimizers.Adam()):
        temp_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_start)
    elif type(optimizer) == type(tf.keras.optimizers.Adamax()):
        temp_optimizer = tf.keras.optimizers.Adamax(learning_rate=lr_start)
    elif type(optimizer) == type(tf.keras.optimizers.Adagrad()):
        temp_optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr_start)
    elif type(optimizer) == type(tf.keras.optimizers.Adadelta()):
        temp_optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr_start)
    elif type(optimizer) == type(tf.keras.optimizers.Nadam()):
        temp_optimizer = tf.keras.optimizers.Nadam(learning_rate=lr_start)
    elif type(optimizer) == type(tf.keras.optimizers.RMSprop()):
        temp_optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_start)
    elif type(optimizer) == type(tf.keras.optimizers.SGD()):
        temp_optimizer = tf.keras.optimizers.SGD(learning_rate=lr_start)
    else:
        print('ERROR in get_model!')
        return None
    model.compile(loss=loss_function, optimizer=temp_optimizer, metrics=[temp_metrics], experimental_run_tf_function=False)
    
    model.summary()
    return model


def fit_model(model, train_generator, validation_generator, iter_dict):
    """
    Fits the model to the train_generator and validates on the validation_generator.
    Returns the history of the model and the stopped epoch.
    """
    
    callback_list = None

    if iter_dict['learning_rate_reduction']:
        learning_rate_reduction = ReduceLROnPlateau(monitor=iter_dict['learning_rate_monitor'],
                                            patience=iter_dict['learning_rate_patience'],
                                            verbose=iter_dict['learning_rate_verbose'],
                                            factor=iter_dict['learning_rate_factor'],
                                            min_lr=iter_dict['learning_rate_min_lr'])
        if callback_list == None:
            callback_list = [learning_rate_reduction]
        else:
            callback_list.append(learning_rate_reduction)
        
    if iter_dict['early_stopping']:
        early_stopping = EarlyStopping(monitor=iter_dict['early_stopping_monitor'],
                                       min_delta=iter_dict['early_stopping_min_delta'], 
                                       patience=iter_dict['early_stopping_patience'], 
                                       verbose=iter_dict['early_stopping_verbose'], 
                                       restore_best_weights=iter_dict['early_stopping_restore_best_weights'])
        if callback_list == None:
            callback_list = [early_stopping]
        else:
            callback_list.append(early_stopping)

    history = model.fit(
        train_generator,
        epochs=iter_dict['epochs'],
        validation_data=validation_generator,
        callbacks=callback_list)
    
    if iter_dict['early_stopping']:
        return history, early_stopping
    else:
        return history, 0

# Grid Search

- get_permutations creates all possible permutations of gs_config. Returns a list of tuples T. Every T consists of tuples t of size 2. The first element of t is the key of the dictionaries and the second element is the value of the dictionaries. See the following example:

    `[(('modelname', 'FarahAmalia'),
    ('optimizer', 'Adam'),
    ('loss_function', 'categorical_crossentropy'),
    ('metric', 'acc'),
    ('epochs', 10),
    ('batch_size', 32)), ...]`


- Function grid_search iterates through all tuples T. In every iteration one we take on T and turn it into a dictionary, that gets returned at the end of the function. One dictionary of this kind includes all the parameters and the corresponding values for one run of grid_search. Example:

    `{'modelname': 'FarahAmalia',
     'optimizer': 'Adam',
     'loss_function': 'categorical_crossentropy',
     'metric': 'acc',
     'epochs': 10,
     'batch_size': 32}`

In [ ]:
def get_permutations(dictionary):
    '''
    Creates all permutations of the input dictionary. 
    Returns these iterations as a list of tupels of tupels.
    '''
    temp_dict = {}
    
    for key, value in dictionary.items():
        temp_dict[key] = [(key, v) for v in value]
    
    elements = list(temp_dict.values())
        
    return list(itertools.product(*elements))

def grid_search():
    """
    For every iteration: get the data and the model corresponding to the settings in that iteration of gs_config.
    Fits the model and returns:
    - best_model_dict: dictionary of best models for different metrics
    - model_result: results of every model
    """
    
    best_model_dict = {
        'val_loss': None,
        'val_categorical_accuracy': None,
        'val_binary_accuracy': None,
        'val_precision': None,
        'val_recall': None
    }
        
    permutations = get_permutations(gs_config)
    labels = os.listdir('./Traindata')

    model_results = []
    for p in permutations:
        iter_dict = dict(p)
        print('\nActual permutation:')
        print(json.dumps(str(iter_dict), indent=1))
        train_gen, val_gen = get_data(iter_dict)

        default_optimizer = iter_dict['optimizer']
        model = get_model(optimizer=default_optimizer, labels_count=len(labels), input_dict=iter_dict)
        history, early_stopping = fit_model(model, train_gen, val_gen, iter_dict)

        # save best models
        for key in best_model_dict.keys():
            if key == 'val_loss' and (best_model_dict[key] is None or min(history.history[key]) < min(best_model_dict[key].history.history[key])):
                    best_model_dict[key] = model
            elif best_model_dict[key] is None or max(history.history[key]) > max(best_model_dict[key].history.history[key]):
                    best_model_dict[key] = model
        model_results.append((iter_dict, history, early_stopping))
       
    return best_model_dict, model_results

# Running grid_search()

In [ ]:
best_models, results  = grid_search()